In [12]:
# original: username,password,session
# strategy, treat username,password as salt, and crack password length
# hash(username,password,session) is known => generate hash(username,password,session,flag)

In [197]:
from math import sin

def get_origin_len(session_id, pw_len):
    origin_seq_len = 7 + pw_len + 2 + len(session_id)
    origin_len = origin_seq_len + 1 # 0x80
    if origin_len % 128 > 120: # pad to next block
        while origin_len % 128 != 0: origin_len += 1
    while origin_len % 128 < 120: origin_len += 1
    origin_len += 8 # sequence length
    return origin_len, origin_seq_len, origin_len - origin_seq_len

def pad_session(session_id, origin_seq_len, origin_pad_len):
    origin_session_id = session_id.encode()
    origin_session_id += bytes([0x80])
    for _ in range(origin_pad_len - 8 - 1):
        origin_session_id += bytes(1)
    origin_session_id += bytes.fromhex(hex(origin_seq_len * 8)[2:].rjust(16, '0'))
    return origin_session_id

def hash_new_seq(known_hash, new_len):
    A = int(known_hash[0:8], 16)
    F = int(known_hash[8:16], 16)
    C = int(known_hash[16:24], 16)
    B = int(known_hash[24:32], 16)
    D = int(known_hash[32:40], 16)
    E = int(known_hash[40:48], 16)
    def G(X,Y,Z):
        return (X ^ (~Z | ~Y) ^ Z) & 0xFFFFFFFF
    def H(X,Y):
        return (X << Y | X >> (32 - Y)) & 0xFFFFFFFF
    X = [int((0xFFFFFFFE) * sin(i)) & 0xFFFFFFFF for i in range(256)]
    
    # hash with new sequence
    s = b"&&flag"
    s += bytes([0x80])
    if len(s) % 128 > 120:
        while len(s) % 128 != 0: s += bytes(1)
    while len(s) % 128 < 120: s += bytes(1)
    s += bytes.fromhex(hex(new_len * 8)[2:].rjust(16, '0'))
    

    for i, b in enumerate(s):
        k, l = int(b), i & 0x1f
        A = (B + H(A + G(B,C,D) + X[k], l)) & 0xFFFFFFFF
        B = (C + H(B + G(C,D,E) + X[k], l)) & 0xFFFFFFFF
        C = (D + H(C + G(D,E,F) + X[k], l)) & 0xFFFFFFFF
        D = (E + H(D + G(E,F,A) + X[k], l)) & 0xFFFFFFFF
        E = (F + H(E + G(F,A,B) + X[k], l)) & 0xFFFFFFFF
        F = (A + H(F + G(A,B,C) + X[k], l)) & 0xFFFFFFFF
    return ''.join(map(lambda x : hex(x)[2:].rjust(8, '0'), [A, F, C, B, D, E]))


r = remote('edu-ctf.csie.org', 42073)
r.sendlineafter("Please Input your username:", b"Guest")
r.recvline() # Hello
session_id = r.recvline().decode().split("Here is your session ID: ")[1].strip()
mac = r.recvline().decode().split("and your MAC(username&&password&&sessionID): ")[1].strip()
print(session_id, mac)

# test_string: Guest&&No FLAG&&session_id{origin padding}&&flag{new padding}
pw_len = 7
origin_len, origin_seq_len, origin_pad_len = get_origin_len(session_id, pw_len)
print(origin_len, origin_seq_len, origin_pad_len)

padded_session_id = pad_session(session_id, origin_seq_len, origin_pad_len)
print(padded_session_id)

new_mac = hash_new_seq(mac, origin_len + 6).encode()
print(new_mac)

payload = bytes.hex(new_mac + b"&&" + padded_session_id + b"&&flag")
r.sendlineafter("What do you want to do? ", payload.encode())
r.recvline()

[x] Opening connection to edu-ctf.csie.org on port 42073
[x] Opening connection to edu-ctf.csie.org on port 42073: Trying 140.112.31.97
[+] Opening connection to edu-ctf.csie.org on port 42073: Done
ec705328f770c1643725 0a418f4ed1f95bef1cc9123205fe81932dfad891ee93fedf
128 36 92
b'ec705328f770c1643725\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01 '
b'2e543eb2f91663eda3a6e5ba51d3a13cceb695f6369b894d'


b'Permission denied\n'

In [198]:
from pwn import *

r = remote('edu-ctf.csie.org', 42073)
r.sendlineafter("Please Input your username:", b"Admin")
r.recvline() # Hello
session_id = r.recvline().decode().split("Here is your session ID: ")[1].strip()
mac = r.recvline().decode().split("and your MAC(username&&password&&sessionID): ")[1].strip()

# target_string: Admin&&pw&&session_id{origin padding}&&flag{new padding}
for i in range(1, 1024):
    pw_len = i
    origin_len, origin_seq_len, origin_pad_len = get_origin_len(session_id, pw_len)
    padded_session_id = pad_session(session_id, origin_seq_len, origin_pad_len)
    new_mac = hash_new_seq(mac, origin_len + 6).encode()

    payload = bytes.hex(new_mac + b"&&" + padded_session_id + b"&&flag")
    r.sendlineafter("What do you want to do? ", payload.encode())
    ret = r.recvline()
    print(i, ret)

[x] Opening connection to edu-ctf.csie.org on port 42073
[x] Opening connection to edu-ctf.csie.org on port 42073: Trying 140.112.31.97
[+] Opening connection to edu-ctf.csie.org on port 42073: Done
1 b'Refused!\n'
2 b'Refused!\n'
3 b'Refused!\n'
4 b'Refused!\n'
5 b'Refused!\n'
6 b'Refused!\n'
7 b'Refused!\n'
8 b'Refused!\n'
9 b'Refused!\n'
10 b'FLAG{c2adf08fdd2baa1e0b0b0caacdcda65c8f5bad36}\n'


EOFError: 